# Thread Cooperation

## Objectives
- How CUDA C calls threads.
- The mechanism for different threads to communicate with each other.
- The mechanism to synchronize the parallel execution of different threads.


###  Splitting Parallel Blocks

**For adding two vectors of size N**

```cpp
add<<<N,1>>>( dev_a, dev_b, dev_c );
```
N blocks x 1 thread/block = N parallel threads

Alternatives - So really, we could have launched N/2 blocks with two threads per block, N/4 blocks with four threads per block.

**For changing the parallel block implementation to a parallel thread implementation**

```cpp
add<<<1,N>>>( dev _ a, dev _ b, dev _ c );
```

For getting the loop id - 
```cpp
int tid = blockIdx.x;  -> int tid = threadIdx.x;
```

#### Vector adition example
```cpp
#include <cuda_runtime.h>

#define HANDLE_ERROR( err )
  if ( err != cudaSuccess ) { 
    fprintf( stderr, "Error: %s\n", cudaGetErrorString( err ) ); 
    exit( EXIT_FAILURE ); 
  }

#define N 10
__global__ void add( int *a, int *b, int *c ) {
    int tid = threadIdx.x;
    if (tid < N)
    c[tid] = a[tid] + b[tid];
}

int main( void ) {
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;
    // allocate the memory on the GPU
    HANDLE_ERROR( cudaMalloc( (void**)&dev_a, N * sizeof(int) ) );
    HANDLE_ERROR( cudaMalloc( (void**)&dev_b, N * sizeof(int) ) );
    HANDLE_ERROR( cudaMalloc( (void**)&dev_c, N * sizeof(int) ) );
    // fill the arrays ‘a’ and ‘b’ on the CPU
    for (int i=0; i<N; i++) {
        a[i] = i;
        b[i] = i * i;
    }
    // copy the arrays ‘a’ and ‘b’ to the GPU
    HANDLE_ERROR( cudaMemcpy( dev_a, a, N * sizeof(int), cudaMemcpyHostToDevice ) );
    HANDLE_ERROR( cudaMemcpy( dev_b, b, N * sizeof(int), cudaMemcpyHostToDevice ) );
    add<<<1,N>>>( dev_a, dev_b, dev_c );
    // copy the array ‘c’ back from the GPU to the CPU
    HANDLE_ERROR( cudaMemcpy( c, dev_c, N * sizeof(int), cudaMemcpyDeviceToHost ) );
    // display the results
    for (int i=0; i<N; i++) {
        printf( “%d + %d = %d\n”, a[i], b[i], c[i] );
    }
    
    // free the memory allocated on the GPU
    cudaFree( dev_a );
    cudaFree( dev_b );
    cudaFree( dev_c );
    return 0;
}

```

#### What if the vector dimension is very big

- The hardware limits the number of blocks in a single launch to 65,535
- The hardware limits the number of threads per block with which we can launch a kernel - 1024(in our case)
- Solution - We will have to use a combination of threads and blocks to accomplish this.
 
```cpp
int tid = threadIdx.x + blockIdx.x * blockDim.x
```

#### What if it is toooooo long

- With our current vector implementation, the upper limit of N is 65,535 * 1024
- Each thread will start at an index given by the following
```cpp 
int tid = threadIdx.x + blockIdx.x * blockDim.x;
```
- After each thread finishes its work at the current index, we need to increment each of them by the total number of threads running in the grid.
- This is simply the number of threads per block multiplied by the number of blocks in the grid, or blockDim.x * gridDim.x
```cpp 
tid += blockDim.x * gridDim.x;
```

-  To ensure we never launch too many blocks, we will just fix the number of blocks to some reasonably small value. 
```cpp
add<<<128,128>>>( dev _ a, dev _ b, dev _ c );
```
- Then just change the vector addition function
```cpp
__global__ void add( int *a, int *b, int *c ) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    while (tid < N) {
        c[tid] = a[tid] + b[tid];
        tid += blockDim.x * gridDim.x;
    }
}
```


## Shared Memory

- The motivation for splitting blocks into threads was simply one of working around hardware limitations to the number of blocks we can have in flight.
- CUDA C makes available a region of memory that we call shared memory.
- You can use the shared memory in the SM processor
    - With keyword
    ```cpp
     _ _shared_ _
    ```
- The CUDA C compiler treats variables in shared memory differently than typical variables. 
- It creates a copy of the variable for each block that you launch on the GPU. 
- Every thread in that block shares the memory, but threads cannot see or modify the copy of this variable that is seen within other blocks.  
- This provides an excellent means by which threads within a block can communicate and collaborate on computations.
- If we expect to communicate between threads, we also need a mechanism forsynchronizing between threads. 
- For example, if thread A writes a value to shared memory and we want thread B to do something with this value, we can’t have thread B start its work until we know the write from thread A is complete. 


### Dot Product example

If we take the dot product of two four-element vectors, we would get

$(x_1,x_2,x_3,x_4)\cdot(y_1,y_2,y_3,y_4) = x_1y_1 + x_2y_2+x_3y_3+x_4y_4$

- Each thread multiplies a pair of corresponding entries, and then every thread moves on to its next pair.
- The result needs to be the sum of all these pairwise products, each thread keeps a running sum of the pairs it has added. 
- Just like in the addition example, the threads increment their indices by the total number of threads to ensure we don’t miss any elements and don’t multiply a pair twice.


```cpp
#define imin(a,b) (a<b?a:b)
const int N = 33 * 1024;
const int threadsPerBlock = 256;
__global__ void dot( float *a, float *b, float *c ) {
    __shared__ float cache[threadsPerBlock];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int cacheIndex = threadIdx.x;
    float temp = 0;
    while (tid < N) {
        temp += a[tid] * b[tid];
        tid += blockDim.x * gridDim.x;
    }
    // set the cache values
    cache[cacheIndex] = temp;
```

- As you can see, we have declared a buffer of shared memory named cache. This buffer will be used to store each thread’s running sum. 
- It will be possible that not every entry will be used if the size of the input vectors is not a multiple of the number of threads per block.
- In this case, the last block will have some threads that do nothing and therefore do not write values.
- At this point in the algorithm, we need to sum all the temporary values we’ve placed in the cache.
- To do this, we will need some of the threads to read the values that have been stored there.
- But we have to make sure this thread will only run after the other threads computes the multiplication

```cpp
// synchronize threads in this block
__syncthreads();

```

- This call guarantees that every thread in the block has completed instructions prior to the __syncthreads() before the hardware will execute the next instruction on any thread. 

- We call the general process of taking an input array and performing some computations that produce a smaller array of results a **reduction**

- The general idea is that each thread will add two of the values in cache[] and store the result back to cache[]. Since each thread combines two entries into one, we complete this step with half as many entries as we started with. In the next step, we do the same thing on the remaining half.

```cpp
// for reductions, threadsPerBlock must be a power of 2
// because of the following code
int i = blockDim.x/2;
while (i != 0) {
    if (cacheIndex < i)
        cache[cacheIndex] += cache[cacheIndex + i];
    __syncthreads();
    i /= 2;
}
```
**One Step of summation reduction**

<img src="fig/one_step.png" width="500"/>

- Observe that we update our shared memory buffer cache[] only if cacheIndex is less than i
- Since cacheIndex is really just threadIdx.x, this means that only some of the threads are updating entries in the shared memory cache. 
- Since we are using __syncthreads only to ensure that these updates have taken place before proceeding, it stands to reason that we might see a speed improvement only if we wait for the threads that are actually writing to shared memory.
